In [9]:
def make_fs_df(firm_code) : 
    
    # URL for the financial statement. I changed the structure of the URL, specifically moving the firm code part to the end of the URL.
    fs_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=D&gicode=A'+firm_code
    
    # Send HTTP request to fetch the page content
    fs_page = requests.get(fs_url)
    
    # Use StringIO to wrap the HTML content and pass it to pd.read_html
    fs_tables = pd.read_html(StringIO(fs_page.text))
    
    # Bring the first financial statement in the website and alter index the first column
    temp_df = fs_tables[0]
    temp_df = temp_df.set_index('IFRS(연결)')
    
    # Change index name 
    temp_df = temp_df.rename_axis('IFRS(Consolidated)')
    
    # Drop 'same FY from previous year' and the ratio column
    temp_df = temp_df.drop(columns=['전년동기', '전년동기(%)'])
    
    # Drop unnessesary rows except Sales, Operating Income and Net income
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]
    
    # Bring Statement of financial position from the web site and set index as 'IFRS(Consolidated)' 
    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index('IFRS(연결)')
    
    # Chane index name from 'IFRS(연결)' to 'IFRS(Consolidated)'
    temp_df2 = temp_df2.rename_axis('IFRS(Consolidated)')
    
    # Drop unnessesary rows except Assets, Liabilities, and Owner's Equity
    temp_df2 = temp_df2.loc[['자산', '부채', '자본']]
    
    # Bring the first financial statement in the website and alter index the first column
    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index('IFRS(연결)')
    
    # Change index name 
    temp_df3 = temp_df3.rename_axis('IFRS(Consolidated)')
    
    # Drop unnessesary rows except Sales, Operating Income and Net income
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]
    
    # Consolidate all the tables above into one table
    fs_df = pd.concat([temp_df, temp_df2, temp_df3])

    # Replace index with new value
    fs_df.index = ['Sales', 'Operating Income', 'Net Income', 'Assets', 'Liabilities', 'Owners Equity', 'Cashflow from Operating Activities']

    return fs_df


In [10]:
def make_fr_df(firm_code) :
    
    # URL for the financial ratio
    fr_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=D&gicode=A'+firm_code
    
    # Send HTTP request to fetch the page content
    fr_page = requests.get(fr_url)
    
    # Use StringIO to wrap the HTML content and pass it to pd.read_html
    fr_tables = pd.read_html(StringIO(fr_page.text))
    
    # Extract the first table from the list of tables
    # The first table in the list is stored in temp_df
    temp_df = fr_tables[0]
    
    # Set the first column as the index of the dataframe
    # The first column is used as the index for better organization and lookup of the rows
    temp_df = temp_df.set_index(temp_df.columns[0])
    
    # Filter the rows of interest based on specific index values
    # Select the rows that correspond to the financial ratios we want to examine (liquidity ratio, debt ratio, etc.)
    temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기', '부채비율계산에 참여한 계정 펼치기', '영업이익률계산에 참여한 계정 펼치기', 'ROA계산에 참여한 계정 펼치기', 'ROIC계산에 참여한 계정 펼치기']]
    
    # Replace index with new value
    temp_df.index = ['Liquidity Ratio', 'Debt Ratio', 'Operatig Income(%)', 'ROA', 'ROIC']

    # Display results
    return temp_df


In [11]:
# Function for making market index data frame
def make_invest_df(firm_code) :
    
    # URL for market index
    invest_url = 'https://comp.fnguide.com/SVO2/ASP/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=&NewMenuID=105&stkGb=D&gicode=A'+firm_code
    
    # Send HTTP request to fetch the page content
    invest_page = requests.get(invest_url)
    
    # Use StringIO to wrap the HTML content and pass it to pd.read_html
    invest_tables = pd.read_html(StringIO(invest_page.text))
        
    # Extract the first table from the list of tables
    temp_df = invest_tables[1]
    
    # Set the first column as the index of the dataframe. 
    temp_df = temp_df.set_index(temp_df.columns[0])
    
    # Select the rows that correspond to the financial ratios we want to examine
    temp_df = temp_df.loc[['EPS계산에 참여한 계정 펼치기(원)', 'PER계산에 참여한 계정 펼치기', 'PBR계산에 참여한 계정 펼치기', 'PCR계산에 참여한 계정 펼치기', 'PSR계산에 참여한 계정 펼치기', '총현금흐름']]
    
    
    # Replace index with new value
    temp_df.index = ['EPS', 'PER', 'PBR', 'PCR', 'PSR', 'Total Cashflow']
    
    
    return temp_df

In [12]:
# Function for changing structure of data frame
def change_df(firm_code, data_frame) : 

    # Create a new DataFrame where the firm code will be the column header for each of the existing columns
    for num, col in enumerate(data_frame.columns) : 

        # Create a temporary DataFrame with the firm code as a column name
        temp_df = pd.DataFrame({firm_code : data_frame[col]})
        
        # Transpose the temporary DataFrame so the columns become rows
        temp_df = temp_df.T

        # Set the new multi-level column index: the first level is the column name from fs_df, the second level is the column name from temp_df
        temp_df.columns = [[col]*len(data_frame), temp_df.columns]

        # If it's the first column, initialize total_df
        if num == 0:
            total_df = temp_df

        else : 
            # Merge the temporary DataFrame with the existing total_df on the index
            total_df = pd.merge(total_df, temp_df, how = 'outer', left_index = True, right_index = True)

            
    return total_df


In [13]:
# Make integrated consolidated financial statements data frame
import requests
import pandas as pd
from io import StringIO

# List of firm codes
firmcode_list = ['A005380', 'A005930', 'A035420', 'A003550', 'A034730']

# Loop through each firm code and get the financial data
for num, code in enumerate(firmcode_list) :
    fs_df = make_fs_df(code)     # Get financial data for each firm
    fs_df_changed = change_df(code, fs_df)     # Change the structure of the data

    # If it's the first initialize total_fs with the first comopany's data
    if num == 0 :
        total_fs = fs_df_changed

    # Otherwise, concatenate the current company's data to total_fs
    else :
        total_fs = pd.concat([total_fs, fs_df_changed])


# Display the consolidated financial statements for all companies
total_fs

2021/12                                                     \
             Sales Operating Income Net Income     Assets Liabilities   
A005380  1176106.0          66789.0    56931.0  2339464.0   1513306.0   
A005930  2796048.0         516339.0   399075.0  4266212.0   1217212.0   
A035420    68176.0          13255.0   164776.0   336910.0     96636.0   
A003550    68590.0          24601.0    26840.0   256698.0     37296.0   
A034730   972037.0          48598.0    57059.0  1653684.0    998157.0   

                                                            2022/12  \
        Owners Equity Cashflow from Operating Activities      Sales   
A005380      826158.0                           -11764.0  1421515.0   
A005930     3048999.0                           651054.0  3022314.0   
A035420      240274.0                            13799.0    82201.0   
A003550      219402.0                            10195.0    71860.0   
A034730      655527.0                            61127.0  1320794.0   

                                     ...     2023/12                \
        Operating Income Net Income  ... Liabilities Owners Equity   
A005380          98249.0    79836.0  ...   1806539.0     1018094.0   
A005930         433766.0   556541.0  ...    922281.0     3636779.0   
A035420          13047.0     6732.0  ...    114998.0      242380.0   
A003550          19414.0    21158.0  ...     32366.0      270165.0   
A034730          81613.0    39662.0  ...   1290912.0      778791.0   

                                              2024/09                   \
        Cashflow from Operating Activities      Sales Operating Income   
A005380                           -25188.0  1286075.0         114174.0   
A005930                           441374.0  2250826.0         262333.0   
A035420                            20022.0    78521.0          14372.0   
A003550                             8827.0    53920.0          12088.0   
A034730                           113539.0   944599.0          25525.0   

                                                         \
        Net Income     Assets Liabilities Owners Equity   
A005380   107558.0  3060866.0   1945726.0     1115140.0   
A005930   266970.0  4913073.0   1050260.0     3862814.0   
A035420    14179.0   373018.0    111465.0      261552.0   
A003550    11017.0   306084.0     29953.0      276131.0   
A034730    23273.0  2094729.0   1277088.0      817641.0   

                                            
        Cashflow from Operating Activities  
A005380                           -17304.0  
A005930                           509604.0  
A035420                            17213.0  
A003550                            10416.0  
A034730                            69383.0  

[5 rows x 28 columns]

In [14]:
# Make integrated financial ratio data frame
import requests
import pandas as pd
from io import StringIO

# List of firm codes
firmcode_list = ['A005380', 'A005930', 'A035420', 'A003550', 'A034730']

# Loop through each firm code and get the financial data
for num, code in enumerate(firmcode_list) :
    fr_df = make_fr_df(code)     # Get financial data for each firm
    fr_df_changed = change_df(code, fr_df)     # Change the structure of the data

    # If it's the first initialize total_fs with the first comopany's data
    if num == 0 :
        total_fr = fr_df_changed

    # Otherwise, concatenate the current company's data to total_fs
    else :
        total_fr = pd.concat([total_fr, fr_df_changed])


# Display the consolidated financial statements for all companies
total_fr

2020/12                                             \
        Liquidity Ratio Debt Ratio Operatig Income(%)   ROA   ROIC   
A005380            85.9      174.2                2.3   1.0    1.9   
A005930           262.2       37.1               15.2   7.2   17.2   
A035420           133.3      106.1               22.9   5.8  171.0   
A003550           276.4       16.4               30.5   6.6   80.6   
A034730           102.3      165.5               -0.1  -0.1    0.9   

                2021/12                                             ...  \
        Liquidity Ratio Debt Ratio Operatig Income(%)   ROA   ROIC  ...   
A005380            83.0      183.2                5.7   2.6    3.8  ...   
A005930           247.6       39.9               18.5   9.9   23.9  ...   
A035420           140.9       40.2               19.4  65.0    NaN  ...   
A003550           196.8       17.0               35.9  10.8  157.0  ...   
A034730           109.4      152.3                5.0   3.8    6.4  ...   

                2023/12                                            \
        Liquidity Ratio Debt Ratio Operatig Income(%)   ROA  ROIC   
A005380            79.9      177.4                9.3   4.6   7.3   
A005930           258.8       25.4                2.5   3.4   3.8   
A035420           111.5       47.5               15.4   2.8  48.6   
A003550           268.4       12.0               21.3   4.7  99.2   
A034730            99.2      165.8                3.4  -0.2   9.2   

                2024/09                                            
        Liquidity Ratio Debt Ratio Operatig Income(%)  ROA   ROIC  
A005380            83.5      174.5                8.9  4.9    6.5  
A005930           251.9       27.2               11.7  7.5   11.7  
A035420           159.5       42.6               18.3  5.2   53.8  
A003550           271.8       10.9               22.4  4.8  102.3  
A034730            94.6      156.2                2.7  1.5    2.6  

[5 rows x 25 columns]

In [15]:
# Make integrated market index data frame
import requests
import pandas as pd
from io import StringIO

# List of firm codes
firmcode_list = ['A005380', 'A005930', 'A035420', 'A003550', 'A034730']

# Loop through each firm code and get the financial data
for num, code in enumerate(firmcode_list) :
    invest_df = make_invest_df(code)     # Get financial data for each firm
    invest_df_changed = change_df(code, invest_df)     # Change the structure of the data

    # If it's the first initialize total_fs with the first comopany's data
    if num == 0 :
        total_invest = invest_df_changed

    # Otherwise, concatenate the current company's data to total_fs
    else :
        total_invest = pd.concat([total_invest, invest_df_changed])


# Display the consolidated financial statements for all companies
total_invest

2020/12                                          2021/12               \
            EPS    PER   PBR    PCR   PSR Total Cashflow     EPS    PER   PBR   
A005380    5144  37.33  0.75   9.48  0.51          63870   17846  11.71  0.75   
A005930    3841  21.09  2.06   9.75  2.32         564882    5777  13.55  1.80   
A035420    6097  47.97  5.60  32.10  9.06          13442  100400   3.77  2.52   
A003550    8334   9.92  0.72   8.87  2.80          14661   15499   5.22  0.61   
A034730    2670  90.08  0.89   2.09  0.21          85535   27592   9.10  0.81   

               ... 2023/12                             2024/09             \
          PCR  ...     PBR    PCR   PSR Total Cashflow     EPS  PER   PBR   
A005380  6.09  ...    0.60   3.30  0.34         161011   37630  NaN  0.65   
A005930  7.24  ...    1.51  10.03  2.06         479071    3834  NaN  1.11   
A035420  3.67  ...    1.51  23.01  3.79          15723    8448  NaN  1.07   
A003550  4.89  ...    0.52   9.50  1.85          15765    6028  NaN  0.47   
A034730  1.73  ...    0.58   1.60  0.10         174800   11567  NaN  0.48   

                                  
         PCR  PSR Total Cashflow  
A005380  NaN  NaN         122154  
A005930  NaN  NaN         547727  
A035420  NaN  NaN          11444  
A003550  NaN  NaN          12017  
A034730  NaN  NaN          85945  

[5 rows x 30 columns]

In [26]:
import pandas as pd

# Load excel workbook for getting company codes listed in Korea market
path = r'C:\\Users\\jongh\\Downloads\\Korea Market Data.xlsx' # Designating file path 

# Read excel file
code_data = pd.read_excel(path)

# Drop the unnessesary columns
code_data = code_data[['영문 종목명', '단축코드']]

# Add a new column with the value 'A' + Code
code_data['Code'] = 'A' + code_data['단축코드']

# Set the company code as index
code_data = code_data.set_index(code_data.columns[2])

# Change column name to Company
code_data = code_data.rename(columns = {'영문 종목명' : 'Company'})

# Drop old company code
code_data = code_data.drop(columns = ['단축코드'])


# Reset index to make 'Code' column available again
code_data_reset = code_data.reset_index()

# Display results
code_data_reset


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U1'), dtype('int64')) -> None

In [21]:
import time # time used for pausing between requests to avoid overwhelming the server or triggering rate-limiting
import requests # requests used for sending HTTP requests to fetch the financial data from an external source (e.g., a website)

# List to store data frames that will be gathered and processed during the loop
dfs = []  # it initializes an empty list to store the individual data frames for each company code. These data frames will later be combined into a single data frame.

# Loop through each company code in the 'Code' column of the 'code_data_reset' DataFrame
for num, code in enumerate(code_data_reset['Code']):  # The loop iterates over the code column in the code_data_reset Dataframe. For each company, it fetches and processes the financial data.
    
    try:
        # Print progress to show the current company code being processed      
        print(f"Processing {num + 1}/{len(code_data_reset['Code'])} - Code: {code}") # It prints the progress of the cript, indicating which company code is being processed and what the progress is.

        # Sleep for 0.1 seconds to avoid making too many requests in a short time (helps prevent throttling or blocking)
        time.sleep(0.1) # This line adds a small delay of 0.1 seconds between requests to avoid making too many requests in a shor time, which could lead to rate-limiting or IP blocking from the server.

        try:
            # Attempt to fetch the financial data for the current company code
            fs_df = make_fs_df(code)  # This part of the codes attempts to fetch the financial data for the current company code using a function make_fs_df().
            

        except requests.exceptions.Timeout:  
        
            # Handle timeout error: if the request times out, wait 60 seconds and then retry fetching the data           
            print(f"Timeout occurred for code {code}. Retrying...")
            
            time.sleep(60)  # Wait for 60 seconds before retrying
            fs_df = make_fs_df(code) # If the request times out, the code waits for 60 seconds and tries again to fetch the data.

        except requests.exceptions.RequestException as e:

            # If there’s a request exception (e.g., network issues, bad response), log the error and move to the next code
            print(f"Request exception occurred for {code}: {e}") # If there's any other issue with the request (e.g., network issue, bad response), the exception is caught, an error message is printed, and the loop moves to the next company code.
            
            continue  # Skip the current iteration and continue with the next company code

        # Check if financial data is returned (fs_df is not None)      
        if fs_df is not None:
            
            # Process the data by changing its structure according to the logic in the 'change_df' function
            fs_df_changed = change_df(code, fs_df)
            
            # Append the processed data frame to the list 'dfs'
            dfs.append(fs_df_changed)

        else:

            # If no financial data is returned for the code, print a message indicating it's skipped           
            print(f"No data returned for {code}, skipping.")

    except (ValueError, KeyError) as e:

        # If there’s an error in processing the data for the code, catch the exception and continue to the next iteration
        print(f"Error processing code {code}: {e}")
        continue # Skip the current code and proceed to the next iteration

# After all iterations, if there are any data frames in the 'dfs' list, concatenate them
if dfs:
    
    # Concatenate all the data frames into one large data frame 'total_fs', row-wise (axis=0)
    total_fs = pd.concat(dfs, axis=0, ignore_index=True)
    print("Successfully consolidated financial data.")

else:
    # If no data frames were added to 'dfs', print a message indicating that no data was available
    print("No financial data available.")

# Display the consolidated financial data in 'total_fs'
total_fs


Processing 1/2865 - Code: A098120
Processing 2/2865 - Code: A009520
Error processing code A009520: "None of ['IFRS(연결)'] are in the columns"
Processing 3/2865 - Code: A095570
Processing 4/2865 - Code: A006840
Processing 5/2865 - Code: A282330
Processing 6/2865 - Code: A027410
Processing 7/2865 - Code: A138930
Error processing code A138930: "['매출액'] not in index"
Processing 8/2865 - Code: A001465
Error processing code A001465: No tables found
Processing 9/2865 - Code: A001460
Processing 10/2865 - Code: A001045
Error processing code A001045: No tables found
Processing 11/2865 - Code: A00104K
Error processing code A00104K: No tables found
Processing 12/2865 - Code: A001040
Processing 13/2865 - Code: A011155
Error processing code A011155: No tables found
Processing 14/2865 - Code: A011150
Processing 15/2865 - Code: A058820
Processing 16/2865 - Code: A000590
Processing 17/2865 - Code: A012030
Processing 18/2865 - Code: A016610
Error processing code A016610: "['매출액'] not in index"
Processing

2021/12                                                                 \
        Sales Operating Income Net Income   Assets Liabilities Owners Equity   
0       614.0             79.0       76.0    516.0        97.0         419.0   
1      9819.0            481.0      767.0  13550.0      9925.0        3625.0   
2     31677.0          -1657.0    -1993.0  45487.0     33464.0       12024.0   
3     67812.0           1994.0     1476.0  25964.0     17871.0        8093.0   
4      1502.0            379.0      360.0  19816.0      2489.0       17328.0   
...       ...              ...        ...      ...         ...           ...   
2017   1418.0            138.0      119.0   1184.0       325.0         859.0   
2018    715.0             67.0       63.0   1319.0       505.0         814.0   
2019    817.0            -19.0      164.0   2473.0      1444.0        1029.0   
2020   2121.0             83.0       62.0   1741.0      1113.0         628.0   
2021    506.0            -44.0       -4.0    843.0       104.0         739.0   

                                         2022/12                              \
     Cashflow from Operating Activities    Sales Operating Income Net Income   
0                                  90.0    611.0             79.0       70.0   
1                                -113.0  11937.0            764.0       87.0   
2                                 -98.0  37880.0           -218.0    -1249.0   
3                                5381.0  76158.0           2524.0     1935.0   
4                                  15.0   4350.0            612.0      262.0   
...                                 ...      ...              ...        ...   
2017                               26.0   1346.0            112.0       86.0   
2018                              184.0    973.0            105.0      119.0   
2019                              -26.0   1779.0            297.0      238.0   
2020                              108.0   2211.0            101.0       71.0   
2021                              -32.0    315.0           -133.0      -86.0   

      ...     2021/08                                                   \
      ... Liabilities Owners Equity Cashflow from Operating Activities   
0     ...         NaN           NaN                                NaN   
1     ...         NaN           NaN                                NaN   
2     ...         NaN           NaN                                NaN   
3     ...         NaN           NaN                                NaN   
4     ...         NaN           NaN                                NaN   
...   ...         ...           ...                                ...   
2017  ...         NaN           NaN                                NaN   
2018  ...         NaN           NaN                                NaN   
2019  ...         NaN           NaN                                NaN   
2020  ...         NaN           NaN                                NaN   
2021  ...         NaN           NaN                                NaN   

     2022/08                                                               \
       Sales Operating Income Net Income Assets Liabilities Owners Equity   
0        NaN              NaN        NaN    NaN         NaN           NaN   
1        NaN              NaN        NaN    NaN         NaN           NaN   
2        NaN              NaN        NaN    NaN         NaN           NaN   
3        NaN              NaN        NaN    NaN         NaN           NaN   
4        NaN              NaN        NaN    NaN         NaN           NaN   
...      ...              ...        ...    ...         ...           ...   
2017     NaN              NaN        NaN    NaN         NaN           NaN   
2018     NaN              NaN        NaN    NaN         NaN           NaN   
2019     NaN              NaN        NaN    NaN         NaN           NaN   
2020     NaN              NaN        NaN    NaN         NaN           NaN   
2021     NaN              NaN        

In [19]:
pip install html5lib

Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install finance-datareader

Note: you may need to restart the kernel to use updated packages.


In [23]:
total_fs.to_excel(r'C:\\Users\\jongh\\Downloads\\Consolidated financial data.xlsx')

In [29]:
import FinanceDataReader as fdr
import pandas as pd
from tqdm.notebook import tqdm

In [33]:
krw_list = fdr.StockListing('KRX')
krw_list = krw_list.set_index(krw_list.columns[0])
krw_list



,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
Code,,,,,,,,,,,,,,,,
005930,KR7005930003,삼성전자,KOSPI,,54100,2,-1200,-2.17,54600,55000,54100,16868600,917178336730,322965235955000,5969782550,STK
000660,KR7000660001,SK하이닉스,KOSPI,,194300,2,-9200,-4.52,203000,203000,193900,4767294,939206630700,141450859519500,728002365,STK
373220,KR7373220003,LG에너지솔루션,KOSPI,,347500,2,-1000,-0.29,346500,350500,345500,165688,57694408000,81315000000000,234000000,STK
207940,KR7207940008,삼성바이오로직스,KOSPI,,1012000,1,12000,1.20,1006000,1044000,1005000,72362,73835125000,72028088000000,71174000,STK
005380,KR7005380001,현대차,KOSPI,,220000,2,-6000,-2.65,225500,226000,218500,867629,191585700000,46071562020000,209416191,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140660,KR7140660002,위월드,KONEX,일반기업부,766,2,-132,-14.70,830,899,766,521,431219,1905235032,2487252,KNX
236030,KR7236030003,씨알푸드,KONEX,일반기업부,930,2,-19,-2.00,930,930,930,2,1860,1890998760,2033332,KNX
413300,KR7413300005,티엘엔지니어링,KONEX,일반기업부,899,2,-1,-0.11,1034,1034,765,26,22142,1215011985,1351515,KNX


In [34]:
import time # time used for pausing between requests to avoid overwhelming the server or triggering rate-limiting
import requests # requests used for sending HTTP requests to fetch the financial data from an external source (e.g., a website)

# List to store data frames that will be gathered and processed during the loop
dfs = []  # it initializes an empty list to store the individual data frames for each company code. These data frames will later be combined into a single data frame.

# Loop through each company code in the 'Code' column of the 'code_data_reset' DataFrame
for num, code in enumerate(krw_list['Code']):  # The loop iterates over the code column in the code_data_reset Dataframe. For each company, it fetches and processes the financial data.
    
    try:
        # Print progress to show the current company code being processed      
        print(f"Processing {num + 1}/{len(krw_list['Code'])} - Code: {code}") # It prints the progress of the cript, indicating which company code is being processed and what the progress is.

        # Sleep for 0.1 seconds to avoid making too many requests in a short time (helps prevent throttling or blocking)
        time.sleep(0.1) # This line adds a small delay of 0.1 seconds between requests to avoid making too many requests in a shor time, which could lead to rate-limiting or IP blocking from the server.

        try:
            # Attempt to fetch the financial data for the current company code
            fs_df = make_fs_df(code)  # This part of the codes attempts to fetch the financial data for the current company code using a function make_fs_df().
            

        except requests.exceptions.Timeout:  
        
            # Handle timeout error: if the request times out, wait 60 seconds and then retry fetching the data           
            print(f"Timeout occurred for code {code}. Retrying...")
            
            time.sleep(60)  # Wait for 60 seconds before retrying
            fs_df = make_fs_df(code) # If the request times out, the code waits for 60 seconds and tries again to fetch the data.

        except requests.exceptions.RequestException as e:

            # If there’s a request exception (e.g., network issues, bad response), log the error and move to the next code
            print(f"Request exception occurred for {code}: {e}") # If there's any other issue with the request (e.g., network issue, bad response), the exception is caught, an error message is printed, and the loop moves to the next company code.
            
            continue  # Skip the current iteration and continue with the next company code

        # Check if financial data is returned (fs_df is not None)      
        if fs_df is not None:
            
            # Process the data by changing its structure according to the logic in the 'change_df' function
            fs_df_changed = change_df(code, fs_df)
            
            # Append the processed data frame to the list 'dfs'
            dfs.append(fs_df_changed)

        else:

            # If no financial data is returned for the code, print a message indicating it's skipped           
            print(f"No data returned for {code}, skipping.")

    except (ValueError, KeyError) as e:

        # If there’s an error in processing the data for the code, catch the exception and continue to the next iteration
        print(f"Error processing code {code}: {e}")
        continue # Skip the current code and proceed to the next iteration

# After all iterations, if there are any data frames in the 'dfs' list, concatenate them
if dfs:
    
    # Concatenate all the data frames into one large data frame 'total_fs', row-wise (axis=0)
    total_fs = pd.concat(dfs, axis=0, ignore_index=True)
    print("Successfully consolidated financial data.")

else:
    # If no data frames were added to 'dfs', print a message indicating that no data was available
    print("No financial data available.")

# Display the consolidated financial data in 'total_fs'
total_fs


KeyError: 'Code'